<a href="https://colab.research.google.com/github/telmacarvalho/tcc-smishing/blob/main/train_ml__smishing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Treinamento dos modelos de *Machine Learning*

## Importa bibliotecas

In [28]:
# Manipulação de dados
import pandas as pd
import joblib

In [29]:
# Machine Learning Clássico (Scikit-Learn)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

## 5. Extração dos dados da pasta *refined*

In [30]:
# Autoriza acesso ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# Extrai os dados refinados da pasta refined
project_path = '/content/drive/MyDrive/tcc/'
file_path = f'{project_path}data/refined/data_processed.csv'
df = pd.read_csv(file_path)
df.head()

,id,source,text,label,comprimento,texto_processado
0,1_0,sms,"Oi, peço para transferir aquele valor para meu...",Legitimate,71,oi peo transfer val mpes nmer
1,2_0,sms,Bom dia babe..tudo bem? Não se esqueça de liga...,Legitimate,85,bom dia babetud bem esque lig pra tio boss h bj
2,3_0,sms,AEN8AFWXJHC Confirmado. Compraste 19.00MT de ...,Legitimate,157,aenafwxjhc confirm compr mt credit pmo nov sal...
3,4_0,sms,"7GD04E51YZM. Caro Cliente, o codigo para efect...",Legitimate,189,gdeyzm car client codig efectu levant cont mpe...
4,5_0,sms,Bom dia babe. Está bem. Vou comprar.\nBoa viagem,Legitimate,47,bom dia bab est bem vou compr boa viag


### 2 Divisão de dados para treino e teste

In [32]:
# Divisão dos dados e vetorização com TF-IDF
X = df['texto_processado']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Limita o vocabulário às 5000 palavras mais frequentes nos dados de treino
vectorizer = TfidfVectorizer(max_features=5000)

# Aprende o vocabulário dos dados de treino (X_train) e os transforma na matriz TF-IDF
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transforma os dados de teste (X_test) usando o vocabulário já aprendido com os dados de treino
# Obs: Não usar fit() nos dados de teste para evitar vazamento de dados (data leakage)
X_test_tfidf = vectorizer.transform(X_test)

print(X_train_tfidf.shape)
print(X_test_tfidf.shape)


(2048, 3349)
(513, 3349)


## 6. Modelagem com *Machine Learning* Clássico

### 6.1  Modelo *Naive Bayes (ComplementNB)*
Usamos uma variação do *Naive Bayes* especificamente projetada para datasets desbalanceados

In [33]:
cnb_model = ComplementNB()
cnb_model.fit(X_train_tfidf, y_train)
y_pred_cnb = cnb_model.predict(X_test_tfidf)

print("Resultados do Complement Naive Bayes:")
print(classification_report(y_test, y_pred_cnb, target_names=['Legítimo', 'Smishing']))

Resultados do Complement Naive Bayes:
              precision    recall  f1-score   support

    Legítimo       1.00      0.93      0.96       402
    Smishing       0.80      0.99      0.89       111

    accuracy                           0.95       513
   macro avg       0.90      0.96      0.93       513
weighted avg       0.96      0.95      0.95       513



### 6.2 Modelo *Random Forest*

Usamos o parâmetro *class_weight='balanced'* para fazer o ajuste de pesos devido ao desbalanceamento do dataset

In [34]:
# Treinamento e Avaliação do Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)
rf_model.fit(X_train_tfidf, y_train)
y_pred_rf = rf_model.predict(X_test_tfidf)

print("Resultados do Random Forest:")
print(classification_report(y_test, y_pred_rf, target_names=['Legítimo', 'Smishing']))

Resultados do Random Forest:
              precision    recall  f1-score   support

    Legítimo       0.99      0.99      0.99       402
    Smishing       0.95      0.95      0.95       111

    accuracy                           0.98       513
   macro avg       0.97      0.97      0.97       513
weighted avg       0.98      0.98      0.98       513



## 7. Armazena os modelos treinados

In [35]:
# Define o caminho da pasta de modelos
models_path = f"{project_path}models/"

In [36]:
# Salva o modelo Complement Naive Bayes
joblib.dump(cnb_model, f"{models_path}complement_naive_bayes.pkl")
print(f"Modelo Naive Bayes salvo com sucesso na pasta: {models_path}")

Modelo Naive Bayes salvo com sucesso na pasta: /content/drive/MyDrive/tcc/models/


In [37]:
# Salva o modelo Random Forest
joblib.dump(rf_model, f"{models_path}random_forest.pkl")
print(f"Modelo Random Forest salvo com sucesso na pasta: {models_path}")

Modelo Random Forest salvo com sucesso na pasta: /content/drive/MyDrive/tcc/models/


In [38]:
# Salva TfidfVectorizer
joblib.dump(vectorizer, f"{models_path}tfidf_vectorizer.pkl")
print(f"Objeto TfidfVectorizer salvo com sucesso na pasta: {models_path}")

Objeto TfidfVectorizer salvo com sucesso na pasta: /content/drive/MyDrive/tcc/models/


## 8. Armazena as métricas

In [39]:
# Define o caminho da pasta de output
outputs_path  = f"{project_path}output/"